In [9]:
%load_ext autoreload
%autoreload 2

## Config file

Only the `config_training.yaml` contained in `./VAD_algorithms/ecovad/` file needs to be updated to run the following pipeline

In [1]:
import yaml
from yaml import FullLoader

# Open the config file
with open("../config_training.yaml") as f:
    cfg = yaml.load(f, Loader=FullLoader)

In [2]:
audio_path = "../assets/demo_data/training_model/soundscape_data/"
speech_dir = "../assets/demo_data/training_model/human_voices/"
noise_dir = "../assets/demo_data/training_model/natural_sounds/"
audio_out_dir = "../assets/demo_data/training_model/synthetic_dataset"


## Generate the training / validation dataset

In [3]:
import glob
from VAD_algorithms.ecovad.make_data import preprocess_file, save_processed_arrays

In [5]:
list_audio_files = glob.glob(audio_path+ "/*")
print("Found {} files to split into training segments".format(len(list_audio_files)))

for file in list_audio_files:
    processed_arr, sr = preprocess_file(file, 
                    cfg["LENGTH_SEGMENTS"], 
                    overlap = 0, 
                    min_length = cfg["LENGTH_SEGMENTS"],
                    speech_dir=speech_dir,
                    noise_dir=noise_dir,
                    proba_speech=cfg["PROBA_SPEECH"],
                    proba_noise_speech=cfg["PROBA_NOISE_WHEN_SPEECH"],
                    proba_noise_nospeech=cfg["PROBA_NOISE_WHEN_NO_SPEECH"])

    save_processed_arrays(file, audio_out_dir, processed_arr, sr)

Found 244 files to split into training segments


/usr/local/lib/python3.8/site-packages/audiomentations/core/audio_loading_utils.py:34: UserWarning: /app/assets/demo_data/training_model/human_voices/163-121908-0011.flac had to be resampled from 16000 hz to 44100 hz. This hurt execution time.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/audiomentations/core/audio_loading_utils.py:34: UserWarning: /app/assets/demo_data/training_model/human_voices/163-121908-0006.flac had to be resampled from 16000 hz to 44100 hz. This hurt execution time.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/audiomentations/core/transforms_interface.py:57: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/usr/local/lib/python3.8/site-packages/audiomentations/core/audio_loading_utils.py:34: UserWarning: /app/assets/demo_data/training_model/human_voices/163-121908-0010.flac had to be resampled from 16000 hz to 44100 hz. This hurt execution time.
  warnings.warn(
/usr/local/lib/python3.8/site-

## Train ecoVAD

In [18]:
train_val_path = "../assets/demo_data/training_model/synthetic_dataset"
ckpt_save_path = "../assets/model_weights/ecoVAD_ckpts_demo.pt"
model_save_path = "../assets/model_weights/ecoVAD_weights_demo.pt"

In [34]:
from VAD_algorithms.ecovad.train_model import trainingApp

trainingApp(train_val_path,
        model_save_path,
        ckpt_save_path,
        cfg["BATCH_SIZE"],
        cfg["NUM_EPOCH"],
        cfg["TB_PREFIX"],
        cfg["TB_COMMENT"],
        cfg["LR"],
        cfg["MOMENTUM"],
        cfg["DECAY"],
        0,
        cfg["USE_GPU"]
        ).main()

Model training on cpu
inputs: tensor([[[[ -60.8932,  -51.1613,  -50.9340,  ...,  -58.9055,  -51.8624,
            -53.5313],
          [ -51.1949,  -42.6964,  -42.3512,  ...,  -51.6650,  -48.2070,
            -43.9337],
          [ -52.7554,  -50.4811,  -45.7739,  ...,  -47.9785,  -49.8013,
            -42.8793],
          ...,
          [ -65.7587,  -65.7587,  -65.7587,  ...,  -65.7587,  -65.7587,
            -65.7587],
          [ -65.7587,  -65.7587,  -65.7587,  ...,  -65.7587,  -65.7587,
            -65.7587],
          [ -65.7587,  -65.7587,  -65.7587,  ...,  -65.7587,  -65.7587,
            -65.7587]]],


        [[[ -53.9401,  -50.0059,  -52.4574,  ...,  -55.9817,  -51.9633,
            -52.1708],
          [ -47.4336,  -47.5363,  -46.1281,  ...,  -49.5115,  -46.0364,
            -48.6834],
          [ -44.5954,  -39.6232,  -42.7425,  ...,  -51.6087,  -45.2783,
            -44.7691],
          ...,
          [ -89.4400,  -85.5805,  -85.8133,  ...,  -86.8414,  -87.0118,
         

KeyboardInterrupt: 